In [1]:
from keras.datasets import cifar10
import numpy as np
np.random.seed(10)

Using TensorFlow backend.


In [2]:
(x_img_train,y_label_train),(x_img_test,y_label_test)=cifar10.load_data()

In [3]:
x_img_train_normalize=x_img_train.astype('float32')/255.0
x_img_test_normalize=x_img_test.astype('float32')/255.0

In [4]:
from keras.utils import np_utils
y_label_train_OneHot=np_utils.to_categorical(y_label_train)
y_label_test_OneHot=np_utils.to_categorical(y_label_test)

In [ ]:
from keras.models import Sequential#导入keras的Sequential模块
from keras.layers import Dense,Dropout,Activation,Flatten#导入的layers模块
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D

In [ ]:
model=Sequential()#建立一个Sequentical线性堆叠模型,后续只需要将模型加入模型即可

In [ ]:
#建立卷积层,卷积的运算的结果类似滤镜效果,可以提取图像的特征
model.add(Conv2D(filters=32,kernel_size=(3,3),#设置随机产生的32个滤镜,每一个滤镜大小3*3
                input_shape=(32,32,3),#图像大小和rgb三色
                activation='relu',#激活函数
                padding='same'))#这个设置让卷积运算产生的图像大小不变

In [ ]:
model.add(Dropout(rate=0.25))#dropout的作用是每次训练迭代时,会随机的在神经网络中放弃25%的神经元,以避免过度拟合

In [ ]:
model.add(MaxPooling2D(pool_size=(2,2)))#缩减采样

In [ ]:
model.add(Conv2D(filters=64,kernel_size=(3,3),
                activation='relu',padding='same'))

In [ ]:
model.add(Dropout(rate=0.25))

In [ ]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
model.add(Conv2D(filters=128,kernel_size=(3,3),
                activation='relu',padding='same'))
model.add(Dropout(rate=0.3))
model.add(Conv2D(filters=128,kernel_size=(3,3),
                activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
model.add(Flatten())
#建立平坦层
model.add(Dropout(rate=0.3))

In [ ]:
model.add(Dense(2500,activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(1500,activation='relu'))
model.add(Dropout(rate=0.3))

In [ ]:
model.add(Dense(10,activation='softmax'))

In [ ]:
print(model.summary())

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])#用compile对训练模型进行设置
#loss设置损失函数,用的是交叉嫡函数,训练的效果比较好
#优化器,adam优化器,可以让训练更加更快收敛,提高准确率
#设置评估模型的方式是准确率

In [ ]:
try:
    model.load_weights("SaveModel/cifarCnnModel.h5")
    print("加载模型成功,继续训练")
except:
    print("加载模型失败,开始训练一个新模型")

In [ ]:
train_history=model.fit(x_img_train_normalize,y_label_train_OneHot,
                       validation_split=0.2,
                       epochs=5,batch_size=300,verbose=1)
#使用model.fit进行训练,训练过程存储在train_history变量中,输入处理后的x和y,设置训练与验证数据比,设置训练周期次数与每一批次项数,执行10个周期,每一批次128项数据

In [ ]:
model.save_weights("SaveModel/cifarCnnModel.h5")
print("Save model to disk")

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
def show_train_history(train_history,train,validation):
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title('Train History')
    plt.ylabel(train)
    plt.xlabel('Epoch')
    plt.legend(['train','validation'],loc='upper left')
    plt.show()
show_train_history(train_history,'accuracy','val_accuracy')

In [ ]:
show_train_history(train_history,'loss','val_loss')

In [ ]:
scores=model.evaluate(x_img_test_normalize,y_label_test_OneHot,verbose=0)#使用model.evaluate模型预测评估模型的准确率,储存在scores
scores[1]

In [ ]:
prediction=model.predict_classes(x_img_test_normalize)
prediction[:10]

In [ ]:
def plot_images_labels_prediction(images,labels,prediction,idx,num=10):
    fig=plt.gcf()
    fig.set_size_inches(12,14)
    if num>25:num=25
    for i in range(0,num):
        ax=plt.subplot(5,5,1+i)
        ax.imshow(images[idx],cmap='binary')
        title= "label=" +str(labels[idx])
        if len(prediction)>0:
            title+=",predict="+str(prediction[idx])
        ax.set_title(title,fontsize=20)
        ax.set_xticks([])
        ax.set_yticks([])
        idx+=1
    plt.show()
plot_images_labels_prediction(x_img_test,y_label_test,prediction,0,10)

In [ ]:
Predicted_Probability=model.predict(x_img_test_normalize)

In [ ]:
label_dict={0:"airplane",1:"automobil",2:"bird",3:"cat",4:"deer",5:"dog",6:"frog",7:"horse",8:"ship",9:"truck"}
def show_Predicted_Probability(y,prediction,x_img,Predictd_Probability,i):
    print('label:',label_dict[y[i][0]],
         'predict',label_dict[prediction[i]])
    plt.figure(figsize=(2,2))
    plt.imshow(np.reshape(x_img_test[i],(32,32,3)))
    plt.show()
    for j in range(10):
        print(label_dict[j]+'Probability:%1.9f'%(Predicted_Probability[i][j]))

In [ ]:
show_Predicted_Probability(y_label_test,prediction,x_img_test,Predicted_Probability,0)

In [ ]:
show_Predicted_Probability(y_label_test,prediction,x_img_test,Predicted_Probability,3)

In [ ]:
prediction.shape

In [ ]:
y_label_test.shape

In [ ]:
y_label_test.reshape(-1)

In [ ]:
import pandas as pd
print(label_dict)
pd.crosstab(y_label_test.reshape(-1),prediction,
           rownames=['label'],colnames=['predict'])